#Notebook para Clasificar Cliente y Recomendador

#Para el caso del Cliente o el sector 

In [1]:
#Se inicializa el notebook generando una conexión con el drive donde se encuentran los CSV
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tqdm import tqdm
#la libreria nltk nos permite utilizar funciones, especialmente las orientadas a NLP, en este caso se utiliza para decargar un complemento del modelo fakenews
import nltk
#La librería pandas nos permite leer, editar, crear y exportar dataframe donde tenemos alojados los datos o donde ponemos los datos obtenidos por el modelo
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import difflib

In [3]:
# Se importan los documentos nesesarios
noticias = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /noticias.csv')
noticias_clientes = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /clientes_noticias.csv')
noticias_inf=noticias_clientes.copy()
noticias_inf=noticias_inf.drop(axis=1, columns=['news_url_absolute', 'news_init_date', 'news_final_date'])
clientes= pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /clientes.csv')

In [8]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
# Se organiza el dataframe noticias_clientes con el fin de poder tener el texto de la noticia, el noommbre de el cliente y el subsec del cliente con los cuales realizaremos
# la clasificacion del cliente
complete= pd.DataFrame()
complete["texto"]=noticias['news_text_content']
complete["news_id"]=noticias['news_id']
resultdf=noticias_inf.merge(complete,how="left",on="news_id")
resultdf = resultdf.merge(clientes, how = 'left',on= "nit")
resultdf=resultdf.drop(axis=1, columns=['desc_ciiu_division', 'desc_ciuu_grupo', 'desc_ciiuu_clase'])
resultdf['texto'] = resultdf['texto'].str.lower()
resultdf['nombre'] = resultdf['nombre'].str.lower()
resultdf['subsec'] = resultdf['subsec'].str.lower()
resultdf = resultdf.dropna().reset_index(drop=True)
resultdf.tail()

,nit,news_id,texto,nombre,subsec
74612,800230209,news99997,"este viernes, el departamento administrativo n...",weatherford colombia limited,extraccion de petroleo
74613,890209174,news99997,"este viernes, el departamento administrativo n...",ismocol sa,extraccion de petroleo
74614,830069311,news99997,"este viernes, el departamento administrativo n...",nabors drilling international ltd bermuda,extraccion de petroleo
74615,830130106,news99997,"este viernes, el departamento administrativo n...",soenergy international colombia sas,gas
74616,890110188,news99997,"este viernes, el departamento administrativo n...",independence drilling sa,extraccion de petroleo


In [11]:
#Se crea una funcion que de los textos me elimina stopwords, sa, sas, puntuacion y numeros con el fin de poder comparar las palabras del nombre y subsec con los textos
Palabras_eliminar = ['sa','sas'] + stopwords.words('spanish') 
def remove_stopwords(sentence, language):
    return [ token for token in nltk.word_tokenize(re.sub(r'[0-9]','', re.sub(r'[^\w\s]', '',sentence ))) if token.lower() not in Palabras_eliminar]

In [ ]:
#Se compara la similitud de palabras que llamaremos keys (Nombre y subsec) con el texto y si supera el 50% de similitud contamos en un vector
Cliente_fid= pd.DataFrame()
for i in tqdm(range(len(resultdf))):
  keys = remove_stopwords(resultdf['nombre'][i],'spanish') + remove_stopwords(resultdf['subsec'][i],'spanish')
  pesos = np.zeros(len(keys))
  text = remove_stopwords(resultdf['texto'][i],'spanish')
  df_text = pd.DataFrame()
  df_text['palabras'] = text
  palabras_comp = df_text['palabras'].value_counts().index
  sumas = df_text['palabras'].value_counts().values
  for j in range(len(keys)):
    for k in range(len(palabras_comp)):
      if difflib.SequenceMatcher(None, keys[j], palabras_comp[k]).ratio() > 0.5:
        pesos[j] = pesos[j]+sumas[k]
  d = {'id':i,'Keys':keys,'weight':pesos, 'total':len(text)}
  df = pd.DataFrame(d)
  Cliente_fid = Cliente_fid.append(df)

100%|██████████| 74617/74617 [1:18:57<00:00, 15.75it/s]


In [ ]:
Cliente_fid.head(20)

,id,Keys,weight,total
0,0,banco,1.0,44
1,0,w,0.0,44
2,0,bancos,2.0,44
0,1,banco,18.0,285
1,1,w,0.0,285
2,1,bancos,23.0,285
0,2,banco,18.0,285
1,2,davivienda,6.0,285
2,2,bancos,23.0,285
0,3,banco,4.0,199


In [ ]:
Cliente_fid.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Grado_cliente.csv', index=False)

In [12]:
Cliente_fid = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Grado_cliente.csv')
Cliente_fid.head()

,id,Keys,weight,total
0,0,banco,1.0,44
1,0,w,0.0,44
2,0,bancos,2.0,44
3,1,banco,18.0,285
4,1,w,0.0,285


In [15]:
#generamos una nueva columna que me divida la cantidad de palabras con respecto al total
Cliente_fid['divide'] = Cliente_fid['weight']/Cliente_fid['total']

In [17]:
#generamos un vvector de suma por id en el cual tendremos en la columna divide la ocurrencia de palabras asociadas al cliente con respecto al total de palabras
Sumado = Cliente_fid.groupby(['id']).sum()

In [18]:
Sumado.head()

,weight,total,divide
id,,,
0,3.0,132,0.068182
1,41.0,855,0.143860
2,47.0,855,0.164912
3,7.0,597,0.035176
4,36.0,1194,0.180905


In [22]:
# teniendo en cuenta que una oracion compleja tiene entre 15 y 20 palabras pero al quitar stopwords y numeros pueden ser entre 5 y 12 y que hay un sujeto que nos interesa pues,
# lo idea es usar almenos un divide de 0.2 que es uno entre 5 pero teniendo en cuenta que el 75% de la distribucion de este caso es un poco mas alta usaremos este valor
Sumado['divide'].describe()

count    74617.000000
mean         0.160336
std          0.084629
min          0.000000
25%          0.100000
50%          0.150358
75%          0.205645
max          1.000000
Name: divide, dtype: float64

In [27]:
# generamos la participacion si es cliente o sector
Sumado['participacion'] = [''] * len(Sumado)
Sumado['participacion'][Sumado['divide']>= 0.205645] = 'Cliente'
Sumado['participacion'][Sumado['divide']< 0.205645] = 'Sector'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
Sumado.drop(columns=['weight','total'])

,divide,participacion
id,,
0,0.068182,Sector
1,0.143860,Sector
2,0.164912,Sector
3,0.035176,Sector
4,0.180905,Sector
...,...,...
74612,0.158879,Sector
74613,0.158879,Sector
74614,0.252336,Cliente


In [34]:
# agregamos este resultado como acercamiento cliente
resultdf['Acercamiento_Cliente'] = Sumado['divide']

In [36]:
# agregamos el label como participacion
resultdf['participacion'] = Sumado['participacion']

In [37]:
resultdf.head()

,nit,news_id,texto,nombre,subsec,Acercamiento_Cliente,participacion
0,900378212,news10006,"este martes, 2 de agosto, el dolar alcanzo un ...",banco w sa,bancos,0.068182,Sector
1,900378212,news10011,no hay nada mas permanente que lo temporal y a...,banco w sa,bancos,0.143860,Sector
2,860034313,news10011,no hay nada mas permanente que lo temporal y a...,banco davivienda sa,bancos,0.164912,Sector
3,900378212,news10015,el equipo de empalme del gobierno del presiden...,banco w sa,bancos,0.035176,Sector
4,900166896,news10015,el equipo de empalme del gobierno del presiden...,hino motors manufacturing colombia sas,equipo de transporte,0.180905,Sector


In [38]:
resultdf.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Participacion_cliente.csv', index=False)

In [17]:
result_df = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Participacion_cliente.csv')

In [18]:
result_df.head()

,nit,news_id,texto,nombre,subsec,Acercamiento_Cliente,participacion
0,900378212,news10006,"este martes, 2 de agosto, el dolar alcanzo un ...",banco w sa,bancos,0.068182,Sector
1,900378212,news10011,no hay nada mas permanente que lo temporal y a...,banco w sa,bancos,0.143860,Sector
2,860034313,news10011,no hay nada mas permanente que lo temporal y a...,banco davivienda sa,bancos,0.164912,Sector
3,900378212,news10015,el equipo de empalme del gobierno del presiden...,banco w sa,bancos,0.035176,Sector
4,900166896,news10015,el equipo de empalme del gobierno del presiden...,hino motors manufacturing colombia sas,equipo de transporte,0.180905,Sector


In [97]:
#Llamamos el archivo de categorizacionn anterior para quitar la sequence agregarr participacion y el nit
cat_ini = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorizacion_inicial.csv')
cat_ini.head()

,news_id,sequence,categoria,nombre_equipo
0,news10006,Precio dolar hoy: la cotización de la divisa a...,reputación,Goofy_Goobers
1,news10011,Es cierto: El presidente Gustavo Petro quiere ...,regulación,Goofy_Goobers
2,news10015,Gustavo Petro: ?Que dice el informe final de s...,regulación,Goofy_Goobers
3,news10028,Cuanto cuesta y como enviar dinero de Estados ...,regulación,Goofy_Goobers
4,news10029,Como enviar dinero de Estados Unidos a otros p...,regulación,Goofy_Goobers


In [98]:
categoria = resultdf.merge(cat_ini, how = 'left',on= "news_id")

In [99]:
categoria = categoria.drop(columns=['texto','nombre','subsec','Acercamiento_Cliente','sequence'])

In [100]:
categoria = categoria[['nombre_equipo','nit','news_id','participacion','categoria']]

In [101]:
categoria.head()

,nombre_equipo,nit,news_id,participacion,categoria
0,Goofy_Goobers,900378212,news10006,Sector,reputación
1,Goofy_Goobers,900378212,news10011,Sector,regulación
2,Goofy_Goobers,860034313,news10011,Sector,regulación
3,Goofy_Goobers,900378212,news10015,Sector,regulación
4,Goofy_Goobers,900166896,news10015,Sector,regulación


In [102]:
# guardamos el archivo final de categorizacion
categoria.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorizacion.csv', index=False)

#Recomendacion
Para la recomendacion tendremos en cuenta los peso proprocionados pero ademas ddos valores los cuales son la probabilidad de la categoria como un score y el acercamiento al cliente (adicional a esto para mejores recomendadores se plantea el uso de la probabilidad de la fake news y un comparador de textos, los cuales se omitieron por la condicion de la base de datos)

In [10]:
score_cat = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorias.csv')
score_cat.head()

,sequence,labels,scores,id,categoria
0,Precio dolar hoy: la cotización de la divisa a...,"[array(['reputación', 'prestigio', 'fama', 'es...","[0.30296555161476135, 0.21928028762340546, 0.1...",news10006,reputación
1,Es cierto: El presidente Gustavo Petro quiere ...,"[array(['reglamentación', 'organización', 'reg...","[0.2925281822681427, 0.2588120996952057, 0.168...",news10011,regulación
2,Gustavo Petro: ?Que dice el informe final de s...,"[array(['reglamentación', 'organización', 'reg...","[0.4224588871002197, 0.1709466427564621, 0.146...",news10015,regulación
3,Cuanto cuesta y como enviar dinero de Estados ...,"[array(['reglamentación', 'organización', 'reg...","[0.29498717188835144, 0.23135842382907867, 0.2...",news10028,regulación
4,Como enviar dinero de Estados Unidos a otros p...,"[array(['reglamentación', 'organización', 'reg...","[0.2932043969631195, 0.20868907868862152, 0.19...",news10029,regulación


In [12]:
#sacamos la probabilidad mas alta que es la primera posicion del vector de la columnna scores
init_score = []
for score in score_cat['scores']:
  init_score.append(score.strip('][').split(', ')[0])
score_cat['init_score'] = init_score 

In [13]:
score_cat['news_id'] = score_cat.id

In [19]:
# realizamos un Join con la base de datos anterior para tener el score y el acercamiento al cliente para cada conbinacion de Nit y noticia
recomendacion_ini = result_df.merge(score_cat, how='left', on='news_id')

In [20]:
recomendacion_ini.head()

,nit,news_id,texto,nombre,subsec,Acercamiento_Cliente,participacion,sequence,labels,scores,id,categoria,init_score
0,900378212,news10006,"este martes, 2 de agosto, el dolar alcanzo un ...",banco w sa,bancos,0.068182,Sector,Precio dolar hoy: la cotización de la divisa a...,"[array(['reputación', 'prestigio', 'fama', 'es...","[0.30296555161476135, 0.21928028762340546, 0.1...",news10006,reputación,0.30296555161476135
1,900378212,news10011,no hay nada mas permanente que lo temporal y a...,banco w sa,bancos,0.143860,Sector,Es cierto: El presidente Gustavo Petro quiere ...,"[array(['reglamentación', 'organización', 'reg...","[0.2925281822681427, 0.2588120996952057, 0.168...",news10011,regulación,0.2925281822681427
2,860034313,news10011,no hay nada mas permanente que lo temporal y a...,banco davivienda sa,bancos,0.164912,Sector,Es cierto: El presidente Gustavo Petro quiere ...,"[array(['reglamentación', 'organización', 'reg...","[0.2925281822681427, 0.2588120996952057, 0.168...",news10011,regulación,0.2925281822681427
3,900378212,news10015,el equipo de empalme del gobierno del presiden...,banco w sa,bancos,0.035176,Sector,Gustavo Petro: ?Que dice el informe final de s...,"[array(['reglamentación', 'organización', 'reg...","[0.4224588871002197, 0.1709466427564621, 0.146...",news10015,regulación,0.4224588871002197
4,900166896,news10015,el equipo de empalme del gobierno del presiden...,hino motors manufacturing colombia sas,equipo de transporte,0.180905,Sector,Gustavo Petro: ?Que dice el informe final de s...,"[array(['reglamentación', 'organización', 'reg...","[0.4224588871002197, 0.1709466427564621, 0.146...",news10015,regulación,0.4224588871002197


In [21]:
recomendacion_ini = recomendacion_ini.drop(columns=['id','texto','nombre','subsec','sequence','participacion','labels','scores'])

In [22]:
recomendacion_ini = recomendacion_ini .dropna().reset_index(drop=True)

In [23]:
recomendacion_ini['categoria'].unique()

array(['reputación', 'regulación', 'alianza', 'macroeconomía',
       'descartable', 'innovación', 'sostenibilidad', 'otros'],
      dtype=object)

In [24]:
recomendacion_ini['init_score'] = recomendacion_ini['init_score'].astype('float')

In [25]:
#asignamos los pesos a las categorias como lo sugiere bancolombia
Pesos = []
for cat in recomendacion_ini['categoria']:
  if cat =='macroeconomía':
    Pesos.append(2)
  if cat =='sostenibilidad':
    Pesos.append(1.8)
  if cat =='innovación':
    Pesos.append(1.7)
  if cat =='regulación':
    Pesos.append(1.6)
  if cat =='alianza':
    Pesos.append(1.4)
  if cat =='reputación':
    Pesos.append(1.2)
  if cat =='otros':
    Pesos.append(0.5)
  if cat =='descartable':
    Pesos.append(0.3)
recomendacion_ini['pesos'] = Pesos

In [26]:
recomendacion_ini.head()

,nit,news_id,Acercamiento_Cliente,categoria,init_score,pesos
0,900378212,news10006,0.068182,reputación,0.302966,1.2
1,900378212,news10011,0.143860,regulación,0.292528,1.6
2,860034313,news10011,0.164912,regulación,0.292528,1.6
3,900378212,news10015,0.035176,regulación,0.422459,1.6
4,900166896,news10015,0.180905,regulación,0.422459,1.6


In [27]:
# utilizamos la los pesos el score y el accercamiento al cliente para calcular el puntaje con el que se hara el recomendador
recomendacion_ini['puntaje'] = recomendacion_ini['init_score']*recomendacion_ini['pesos']+recomendacion_ini['Acercamiento_Cliente']

In [28]:
#Organizamos el dataframe para que los puntajes mas altos queden primero
recomendacion_ini = recomendacion_ini.sort_values(by=['nit', 'puntaje'], ascending=False).reset_index(drop=True)

In [29]:
recomendacion_ini

,nit,news_id,Acercamiento_Cliente,categoria,init_score,pesos,puntaje
0,901576411,news11461,0.078947,alianza,0.743764,1.4,1.120217
1,901576411,news79518,0.108108,alianza,0.694054,1.4,1.079784
2,901576411,news96711,0.090343,alianza,0.623128,1.4,0.962722
3,901576411,news79511,0.071895,regulación,0.524529,1.6,0.911141
4,901576411,news98672,0.130435,alianza,0.548648,1.4,0.898542
...,...,...,...,...,...,...,...
74612,800000946,news18687,0.318493,descartable,0.217730,0.3,0.383812
74613,800000946,news97860,0.243421,descartable,0.249550,0.3,0.318286
74614,800000946,news89860,0.225225,descartable,0.240338,0.3,0.297327
74615,800000946,news89859,0.226115,descartable,0.227142,0.3,0.294257


In [35]:
#Asignamos las posiciones de recomendacion y las aderimos al dataframe
Recom = []
for n in recomendacion_ini['nit'].unique():
  Recom = Recom + list(np.arange(1,len(recomendacion_ini[recomendacion_ini['nit']==n])+1))
recomendacion_ini['recomendacion'] = Recom

In [37]:
recomendacion_ini.head(20)

,nit,news_id,Acercamiento_Cliente,categoria,init_score,pesos,puntaje,recomendacion
0,901576411,news11461,0.078947,alianza,0.743764,1.4,1.120217,1
1,901576411,news79518,0.108108,alianza,0.694054,1.4,1.079784,2
2,901576411,news96711,0.090343,alianza,0.623128,1.4,0.962722,3
3,901576411,news79511,0.071895,regulación,0.524529,1.6,0.911141,4
4,901576411,news98672,0.130435,alianza,0.548648,1.4,0.898542,5
5,901576411,news94565,0.149606,regulación,0.409899,1.6,0.805445,6
6,901576411,news63086,0.077922,alianza,0.513683,1.4,0.797078,7
7,901576411,news31020,0.098446,regulación,0.415996,1.6,0.764040,8
8,901576411,news98678,0.090361,regulación,0.387481,1.6,0.710331,9
9,901576411,news96902,0.162162,regulación,0.324411,1.6,0.681220,10


In [38]:
#ahora solo nesesitamos el dataframe anterior para agregarle la recomendacion
recomendacion = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorizacion_final.csv')
recomendacion.head()

,nombre_equipo,nit,news_id,participacion,categoria
0,Goofy_Goobers,900378212,news10006,Sector,reputación
1,Goofy_Goobers,900378212,news10011,Sector,regulación
2,Goofy_Goobers,860034313,news10011,Sector,regulación
3,Goofy_Goobers,900378212,news10015,Sector,regulación
4,Goofy_Goobers,900166896,news10015,Sector,regulación


In [41]:
#unimos llos anteriores con tal de que para cada combinacion nit noticia tengamos los demas valores
recomendacion_final = pd.merge(recomendacion, recomendacion_ini,  how='left', left_on=['nit','news_id'], right_on = ['nit','news_id']).sort_values(by=['nit', 'puntaje'], ascending=False).reset_index(drop=True)

In [43]:
# eliminamos las columnas innesesarias y adecuamos el sistema a lo previsto por el reto
recomendacion_final = recomendacion_final.drop(columns=['categoria_y','init_score','Acercamiento_Cliente','pesos','puntaje'])
recomendacion_final = recomendacion_final.rename(columns={"categoria_x": "categoria"})

In [49]:
recomendacion_final.head(50)

,nombre_equipo,nit,news_id,participacion,categoria,recomendacion
0,Goofy_Goobers,901576411,news11461,Sector,alianza,1
1,Goofy_Goobers,901576411,news79518,Sector,alianza,2
2,Goofy_Goobers,901576411,news96711,Sector,alianza,3
3,Goofy_Goobers,901576411,news79511,Sector,regulación,4
4,Goofy_Goobers,901576411,news98672,Sector,alianza,5
5,Goofy_Goobers,901576411,news94565,Sector,regulación,6
6,Goofy_Goobers,901576411,news63086,Sector,alianza,7
7,Goofy_Goobers,901576411,news31020,Sector,regulación,8
8,Goofy_Goobers,901576411,news98678,Sector,regulación,9
9,Goofy_Goobers,901576411,news96902,Sector,regulación,10


In [48]:
#guardamos el archivo de recomendacion
recomendacion_final.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /recomendacion.csv', index=False)